In [9]:
import pandas as pd
import numpy as np

# --- 1. Load Data ---
# Load all three datasets. Note the special delimiter for the GDP file.
df_emissions = pd.read_csv("co-emissions-by-sector.csv")
df_pop = pd.read_csv("world_population.csv")

print("Step 1: Data Loaded Successfully.")

Step 1: Data Loaded Successfully.


In [11]:
# --- 2. PREPARE EMISSIONS DATA (Calculate Total Emissions) ---

# Identify all columns that represent a sector's CO2 emission
emission_cols = [col for col in df_emissions.columns if 'emissions from' in col]

# Sum all sector columns row-wise to get the total
df_emissions['Total Emissions'] = df_emissions[emission_cols].sum(axis=1)

# Cleaned emissions columns (excluding Fugitive, as it contains many NaNs)
clean_emission_cols = [
    'Carbon dioxide emissions from buildings',
    'Carbon dioxide emissions from industry',
    'Carbon dioxide emissions from land use change and forestry',
    'Carbon dioxide emissions from other fuel combustion',
    'Carbon dioxide emissions from transport',
    'Carbon dioxide emissions from manufacturing and construction',
    'Carbon dioxide emissions from electricity and heat',
    'Carbon dioxide emissions from bunker fuels'
]
df_emissions_clean = df_emissions[['Entity', 'Year', 'Code', 'Total Emissions'] + clean_emission_cols].copy()
print("Step 2: Data cleaned Successfully.")

Step 2: Data cleaned Successfully.


In [12]:
# 3. PREPARE POPULATION DATA (Unpivot and Clean) ---


# Identify all 'YYYY Population' columns to be unpivoted
pop_cols_to_melt = [col for col in df_pop.columns if 'Population' in col and col != 'World Population Percentage']

# Reshape the data from WIDE (years as columns) to LONG (Year and Population columns)
df_pop_long = pd.melt(
    df_pop,
    id_vars=['Country/Territory'], # The column to keep fixed
    value_vars=pop_cols_to_melt,
    var_name='Year',
    value_name='Population'
)

# Clean Columns for Merging
#
df_pop_long.rename(columns={'Country/Territory': 'Entity'}, inplace=True)
# Extract the year number and convert to integer for proper merging
df_pop_long['Year'] = df_pop_long['Year'].str.replace(' Population', '').astype(int)
print("Step 3: Population Data Reshaped and Cleaned.")



Step 3: Population Data Reshaped and Cleaned.


In [14]:
# --- 4. MERGE DATAFRAMES AND CALCULATE FINAL METRIC ---
# We still need a minimal mapping to catch major mismatches between emissions and population files
pop_name_map = {
    'Egypt, Arab Rep.': 'Egypt',
    'Russian Federation': 'Russia',
    'Korea, Rep.': 'South Korea',
    'Iran, Islamic Rep.': 'Iran',
    'Congo, Dem. Rep.': 'Democratic Republic of Congo',
    'Yemen, Rep.': 'Yemen'
}
df_pop_long['Entity'] = df_pop_long['Entity'].replace(pop_name_map)


df_final = df_emissions_clean.merge(
    df_pop_long[['Entity', 'Year', 'Population']],
    on=['Entity', 'Year'],
    how='left'
)

# Replace zero population values with NaN to avoid division errors
df_final['Population'] = df_final['Population'].replace(0, np.nan)

# Calculate the single required metric
df_final['Emissions per Capita'] = df_final['Total Emissions'] / df_final['Population']
print("Step 4: merge succesfully")


Step 4: merge succesfully


In [16]:
# --- 5. FINAL SAVE ---
df_final = df_final[df_final['Total Emissions'] > 0]
final_file_name = 'final_dashboard_data.csv'
df_final.to_csv(final_file_name, index=False)

print(f"Data preparation complete . File saved as '{final_file_name}'.")

Data preparation complete . File saved as 'final_dashboard_data.csv'.
